In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# train data loading
# 이미지 - 배열 변환후 csv로 출력
for n in range(10):
    print(n)
    df = list()
    for i in range(n*625, (n+1)*625):
        # data loading & resize
        dog_img = Image.open(f'../data/dogs-vs-cats/train/dog.{i}.jpg').resize((150,150))
        cat_img = Image.open(f'../data/dogs-vs-cats/train/cat.{i}.jpg').resize((150,150))

        # 흑백변환
        dog_img = np.mean(dog_img, axis=2, keepdims=True, dtype=np.int32).astype(np.float32)
        cat_img = np.mean(cat_img, axis=2, keepdims=True, dtype=np.int32).astype(np.float32)

        df.append([1] + dog_img.ravel().tolist())
        df.append([0] + cat_img.ravel().tolist())

    df = pd.DataFrame(df)
    df.to_csv(f'../data/dogs-vs-cats/batch{n}.csv', index=False)
    del df

In [2]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), 
                 activation='relu', input_shape=(150,150,1), padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), 
                 activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), 
                 activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), 
                 activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)       1

In [16]:
# learning
model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_list = list()
for n in range(10):
    # data load
    df = pd.read_csv(f'../data/dogs-vs-cats/batch{n}.csv')
    # data shuffle
    df = df.sample(frac=1).reset_index(drop=True).values
    
    # x, t data split
    x_data = df[:,1:].copy()
    t_data = df[:,0].copy()
    del df
    
    # normalization
    x_data = x_data % 255
    
    # reshape
    x_data = x_data.reshape(-1, 150, 150, 1)
    
    print(f'{n+1}번째 학습 시작')
    history_list.append(model.fit(x_data, t_data, 
                        epochs=30, verbose=0, validation_split=0.3))
    print(f'{n+1}번째 학습 종료')
    del x_data
    del t_data

1번째 학습 시작
1번째 학습 종료
2번째 학습 시작
2번째 학습 종료
3번째 학습 시작
3번째 학습 종료
4번째 학습 시작
4번째 학습 종료
5번째 학습 시작
5번째 학습 종료
6번째 학습 시작
6번째 학습 종료
7번째 학습 시작
7번째 학습 종료
8번째 학습 시작
8번째 학습 종료
9번째 학습 시작
9번째 학습 종료
10번째 학습 시작
10번째 학습 종료


In [23]:
acc_list = list()
for history in history_list:
    acc = np.mean(history.history['accuracy'])
    print(acc)
    acc_list.append(acc)

print(f'Total Accuracy : {np.mean(acc_list)}')

0.9085333307584127
0.5736000001430511
0.8858666698137919
0.7466285745302836
0.8937523821989696
0.8470476170380911
0.904152375459671
0.8520380934079488
0.8661714295546213
0.9244571407636006
Total Accuracy : 0.8402247613668443


In [14]:
# data shuffle test

df = pd.read_csv('../data/dogs-vs-cats/batch0.csv')
df = df.sample(frac=1).reset_index(drop=True).values

# x, t data split
x_data = df[:,1:].copy()
t_data = df[:,0].copy()

In [ ]:
%reset
# 일부 이미지 분리(총 4000개)

import os, shutil

original_dataset_dir = '../data/dogs-vs-cats/train'

## directory 생성 ##

base_dir = '../data/cat_dog_small'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir,'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

train_cats_dir = os.path.join(train_dir,'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir,'dogs')
os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(validation_dir,'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir,'dogs')
os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir,'cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir,'dogs')
os.mkdir(test_dogs_dir)

## file 복사 ##

fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src,dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src,dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src,dst)
    

fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src,dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src,dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src,dst)